In [6]:
#Libraries
import pypyodbc
import pandas as pd
#Conn and credentials
conn_str = 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:servidortcprueba.database.windows.net,1433;Database=db_campechanos;Uid=adminraul;Pwd=5ccu9$j$XzhNMMF;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
connection = pypyodbc.connect(conn_str)
cursor = connection.cursor()

#Training dataset
query = "SELECT * FROM train"
cursor.execute(query)
rows = cursor.fetchall()
df_train = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_train.columns = columns

#Test dataset, dont have sales price
query = "SELECT * FROM test"
cursor.execute(query)
rows = cursor.fetchall()
df_test = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_test.columns = columns

#Sample dataset, id and sales price
query = "SELECT * FROM sample"
cursor.execute(query)
rows = cursor.fetchall()
df_sample = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_sample.columns = columns

Exception ignored in: <function Connection.__del__ at 0x00000183C238B010>
Traceback (most recent call last):
  File "c:\Users\rauli\anaconda3\envs\IA\lib\site-packages\pypyodbc.py", line 2682, in __del__
    self.close()
  File "c:\Users\rauli\anaconda3\envs\IA\lib\site-packages\pypyodbc.py", line 2695, in close
    self.rollback()
  File "c:\Users\rauli\anaconda3\envs\IA\lib\site-packages\pypyodbc.py", line 2618, in rollback
    check_success(self, ret)
  File "c:\Users\rauli\anaconda3\envs\IA\lib\site-packages\pypyodbc.py", line 1009, in check_success
    ctrl_err(SQL_HANDLE_DBC, ODBC_obj.dbc_h, ret, ODBC_obj.ansi)
  File "c:\Users\rauli\anaconda3\envs\IA\lib\site-packages\pypyodbc.py", line 987, in ctrl_err
    raise DatabaseError(state,err_text)
pypyodbc.DatabaseError: ('08S01', '[08S01] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Se ha forzado la interrupción de una conexión existente por el host remoto.\r\n')


In [7]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
df_test=df_test.drop('na_mszoning',axis=1)
df_test= df_test.drop('na_saletype',axis=1)
columns_in_test = df_test.columns

# Filtra las columnas en df_train que también están en df_test
df_train_filtered = df_train[columns_in_test]

In [9]:
X_train = df_train.drop('saleprice', axis=1)  # Ajusta 'target_column' a tu columna objetivo real
y_train = df_train['saleprice']

X_train=X_train.drop('is_house_25fstory',axis=1)
columns_in_train = X_train.columns

# Filtra las columnas en df_test que también están en df_train
df_test_filtered = df_test[columns_in_train]

In [13]:
import random
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

X_test = df_test_filtered
y_test = df_sample['saleprice']

best_mse = float('inf')  # Inicializa el mejor MSE con un valor infinito
best_r2 = -float('inf')  # Inicializa el mejor R^2 con un valor negativo infinito
best_pred = None

max_tries = 100
params = {
    'n_estimators': 100,
    'max_depth': 5,
    'learning_rate': 0.1,
    'random_state': 0
}

condition = False
count=1

r2_cond= -20
mse_cond= 504500003155.638916

max_tries=range(0,50)

for _ in max_tries:
    # Actualiza los parámetros con valores aleatorios
    params['n_estimators'] = random.randint(10, 1000)
    params['max_depth'] = random.randint(3, 1000)
    params['learning_rate'] = random.uniform(0.005, 1.0)
    params['random_state'] = random.randint(0, 1000)

    # Crea y entrena el modelo de árbol de regresión con los parámetros aleatorios
    regression_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        random_state=params['random_state']
    )
    regression_model.fit(X_train, y_train)

    # Realiza predicciones en el conjunto de prueba
    y_pred = regression_model.predict(X_test)

    # Evalúa el rendimiento del modelo
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    if mse<best_mse:
        best_pred = y_pred
        best_r2 = r2
        best_mse = mse
        best_params = params.copy()

print("Mejores hiperparámetros:")
print(best_params)
print(f"Error cuadrático medio (MSE): {best_mse}")
print(f"Coeficiente de determinación (R^2): {best_r2}")

KeyboardInterrupt: 

In [11]:
past_r2= best_r2
past_mse = best_mse
past_pred=best_pred
print(f"Error cuadrático medio (MSE): {past_mse/1000000000}")
print(f"Coeficiente de determinación (R^2): {past_r2}")

Error cuadrático medio (MSE): 4.884620815248097
Coeficiente de determinación (R^2): -16.91421427509967


In [14]:
#More Hiperparameters
import random
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

X_test = df_test_filtered
y_test = df_sample['saleprice']

best_mse_hip = float('inf')  # Inicializa el mejor MSE con un valor infinito
best_r2_hip = -float('inf')  # Inicializa el mejor R^2 con un valor negativo infinito
best_pred_hip = None

max_tries = 50

for _ in range(max_tries):
    # Genera valores aleatorios para todos los hiperparámetros posibles
    params = {
        'n_estimators': random.randint(10, 1000),
        'max_depth': random.randint(3, 100),
        'learning_rate': random.uniform(0.001, 1.0),
        'subsample': random.uniform(0.5, 1.0),
        'colsample_bytree': random.uniform(0.5, 1.0),
        'gamma': random.uniform(0, 1.0),
        'reg_alpha': random.uniform(0, 1.0),
        'reg_lambda': random.uniform(0, 1.0),
        'min_child_weight': random.uniform(1, 10),
        'random_state': random.randint(0, 1000)
    }

    # Crea y entrena el modelo de árbol de regresión con los parámetros aleatorios
    regression_model = xgb.XGBRegressor(**params, objective='reg:squarederror')
    regression_model.fit(X_train, y_train)

    # Realiza predicciones en el conjunto de prueba
    y_pred = regression_model.predict(X_test)

    # Evalúa el rendimiento del modelo
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    if mse < best_mse_hip:
        best_pred_hip = y_pred
        best_r2_hip = r2
        best_mse_hip = mse
        best_params_hip = params.copy()

print("Mejores hiperparámetros:")
print(best_params_hip)
print(f"Error cuadrático medio (MSE): {best_mse_hip/1000000000}")
print(f"Coeficiente de determinación (R^2): {best_r2_hip}")


Mejores hiperparámetros:
{'n_estimators': 77, 'max_depth': 40, 'learning_rate': 0.22814120619150743, 'subsample': 0.806824415097525, 'colsample_bytree': 0.5104388057298872, 'gamma': 0.8288938864702186, 'reg_alpha': 0.9945293821785826, 'reg_lambda': 0.28198489972784424, 'min_child_weight': 2.302200931051233, 'random_state': 643}
Error cuadrático medio (MSE): 5.079498413271502
Coeficiente de determinación (R^2): -17.628922577023552


In [19]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Define el rango de random_state que deseas probar
random_state_range = range(1, 101)  # Por ejemplo, prueba valores de 1 a 100

best_model = None
best_score = -float('inf')  # Inicializa el mejor score con un valor negativo infinito

for random_state in random_state_range:
    # Crea un modelo XGBoost con el valor actual de random_state
    model = xgb.XGBRegressor(random_state=random_state)
    model.fit(X_train, y_train)

    # Calcula el score del modelo en el conjunto de prueba
    model_score = model.score(X_test, y_test)

    # Actualiza el mejor modelo si se encuentra un modelo con un mejor score
    if model_score > best_score:
        best_score = model_score
        best_model = model
        best_r = random_state

# Realiza predicciones en el conjunto de prueba usando el mejor modelo
best_model_pred = best_model.predict(X_test)

# Calcula el MSE y R^2 del mejor modelo
best_mse = mean_squared_error(y_test, best_model_pred)
best_r2 = r2_score(y_test, best_model_pred)

# Imprime el mejor score y las métricas del mejor modelo
print(f"Mejor Score del modelo: {best_score}")
print(f"Mejor Error cuadrático medio (MSE): {best_mse}")
print(f"Mejor Coeficiente de determinación (R^2): {best_r2}")
print(f"Mejor Random: {best_r}")

Mejor Score del modelo: -18.396516091260402
Mejor Error cuadrático medio (MSE): 5288796080.459861
Mejor Coeficiente de determinación (R^2): -18.396516091260402
Mejor Random: 1


In [15]:
df_predictions = pd.DataFrame({'id': df_sample['id'], 'SalePrice': best_model_pred})

# Guarda el DataFrame resultante en un archivo CSV
df_predictions.to_csv('Ppredicciones.csv', index=False)